# Sudoku

Copyright Luca de Alfaro, 2019-20. 
License: [CC-BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/4.0/).


Prepared on: Tue Nov 23 07:54:36 2021

This is a book chapter; it is not a homework assignment.  
Do not submit it as a solution to a homework assignment; you would receive no credit.


Let us write a [Sudoku](https://en.wikipedia.org/wiki/Sudoku) solver.  We want to get as input a Sudoku with some cells filled with values, and we want to get as output a solution, if one exists, and otherwise a notice that the input Sudoku puzzle has no solutions. 

You will wonder, why spend so much time on Sudoku? 

For two reasons. 

First, the way we go about solving Sudoku is prototypical of a very large number of problems in computer science.  In these problems, the solution is attained through a mix of search (we attempt to fill a square with a number and see if it works out), and constraint propagation (if we fill a square with, say, a 1, then there can be no 1's in the same row, column, and 3x3 square).

Second, and related, the way we go about solving Sudoku puzzles is closely related to how [SAT solvers](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem#Algorithms_for_solving_SAT) work.  So closely related, in fact, that while _we_ describe for you how a Sudoku solver works, _you_ will have to write a SAT solver as exercise. 


## Sudoku representation

First, let us do some grunt work and define a representation for a Sudoku problem. 

One initial idea would be to represent a Sudoku problem via a $9 \times 9$ matrix, where each entry can be either a digit from 1 to 9, or 0 to signify "blank".  This would work in some sense, but it would not be a very useful representation.  If you have solved Sudoku by hand (and if you have not, please go and solve a couple; it will teach you a lot about what we need to do), you will know that the following strategy works: 

Repeat: 
* Look at all blank spaces.  Can you find one where only one digit fits? If so, write the digit there. 
* If you cannot find any blank space as above, try to find one where only a couple or so digits can fit.  Try putting in one of those digits, and see if you can solve the puzzle with that choice.  If not, backtrack, and try another digit. 

Thus, it will be very useful to us to remember not only the known digits, but also, which digits can fit into any blank space. 
Hence, we represent a Sudoku problem via a $9 \times 9$ matrix of _sets_: each set contains the digits that can fit in a given space. 
Of course, a known digit is just a set containing only one element. 
We will solve a Sudoku problem by progressively "shrinking" these sets of possibilities, until they all contain exactly one element. 

Let us write some code that enables us to define a Sudoku problem, and display it for us; this will be very useful both for our fun and for debugging. 


First, though, let's write a tiny helper function that returns the only element from a singleton set.

In [ ]:
def getel(s):
    """Returns the unique element in a singleton set (or list)."""
    assert len(s) == 1
    return list(s)[0]


In [ ]:
import json

class Sudoku(object):

    def __init__(self, elements):
        """Elements can be one of:
        Case 1: a list of 9 strings of length 9 each.
        Each string represents a row of the initial Sudoku puzzle,
        with either a digit 1..9 in it, or with a blank or _ to signify
        a blank cell.
        Case 2: an instance of Sudoku.  In that case, we initialize an
        object to be equal (a copy) of the one in elements.
        Case 3: a list of list of sets, used to initialize the problem."""
        if isinstance(elements, Sudoku):
            # We let self.m consist of copies of each set in elements.m
            self.m = [[x.copy() for x in row] for row in elements.m]
        else:
            assert len(elements) == 9
            for s in elements:
                assert len(s) == 9
            # We let self.m be our Sudoku problem, a 9x9 matrix of sets.
            self.m = []
            for s in elements:
                row = []
                for c in s:
                    if isinstance(c, str):
                        if c.isdigit():
                            row.append({int(c)})
                        else:
                            row.append({1, 2, 3, 4, 5, 6, 7, 8, 9})
                    else:
                        assert isinstance(c, set)
                        row.append(c)
                self.m.append(row)


    def show(self, details=False):
        """Prints out the Sudoku matrix.  If details=False, we print out
        the digits only for cells that have singleton sets (where only
        one digit can fit).  If details=True, for each cell, we display the
        sets associated with the cell."""
        if details:
            print("+-----------------------------+-----------------------------+-----------------------------+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    # We represent the set {2, 3, 5} via _23_5____
                    s = ''
                    for k in range(1, 10):
                        s += str(k) if k in self.m[i][j] else '_'
                    r += s
                    r += '|' if (j + 1) % 3 == 0 else ' '
                print(r)
                if (i + 1) % 3 == 0:
                    print("+-----------------------------+-----------------------------+-----------------------------+")
        else:
            print("+---+---+---+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    if len(self.m[i][j]) == 1:
                        r += str(getel(self.m[i][j]))
                    else:
                        r += "."
                    if (j + 1) % 3 == 0:
                        r += "|"
                print(r)
                if (i + 1) % 3 == 0:
                    print("+---+---+---+")


    def to_string(self):
        """This method is useful for producing a representation that
        can be used in testing."""
        as_lists = [[list(self.m[i][j]) for j in range(9)] for i in range(9)]
        return json.dumps(as_lists)


    @staticmethod
    def from_string(s):
        """Inverse of above."""
        as_lists = json.loads(s)
        as_sets = [[set(el) for el in row] for row in as_lists]
        return Sudoku(as_sets)


    def __eq__(self, other):
        """Useful for testing."""
        return self.m == other.m


Let us input a problem (the Sudoku example found on [this Wikipedia page](https://en.wikipedia.org/wiki/Sudoku)) and check that our serialization and deserialization works.

In [ ]:
# Let us ensure that nose is installed.
try:
    from nose.tools import assert_equal, assert_true
    from nose.tools import assert_false, assert_almost_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_true
    from nose.tools import assert_false, assert_almost_equal


     |████████████████████████████████| 163kB 5.9MB/s 


In [ ]:
from nose.tools import assert_equal

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.show(details=True)
s = sd.to_string()
sdd = Sudoku.from_string(s)
sdd.show(details=True)
assert_equal(sd, sdd)


+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 123456789|123456789 ______7__ 123456789|123456789 123456789 123456789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+---------------------

Let's test our constructor statement when passed a Sudoku instance.

In [ ]:
sd1 = Sudoku(sd)
assert_equal(sd, sd1)


## Constraint propagation

When the set in a Sudoku cell contains only one element, this means that the digit at that cell is known. 
We can then propagate the knowledge, ruling out that digit in the same row, in the same column, and in the same 3x3 cell. 

We first write a method that propagates the constraint from a single cell.  The method will return the list of newly-determined cells, that is, the list of cells who also now (but not before) are associated with a 1-element set.  This is useful, because we can then propagate the constraints from those cells in turn.  Further, if an empty set is ever generated, we raise the exception Unsolvable: this means that there is no solution to the proposed Sudoku puzzle. 

We don't want to steal all the fun from you; thus, we will give you the main pieces of the implemenetation, but we ask you to fill in the blanks.  We provide tests so you can catch any errors right away.

### Propagating a single cell

In [ ]:
class Unsolvable(Exception):
    pass


def sudoku_ruleout(self, i, j, x):
    """The input consists in a cell (i, j), and a value x.
    The function removes x from the set self.m[i][j] at the cell, if present, and:
    - if the result is empty, raises Unsolvable;
    - if the cell used to be a non-singleton cell and is now a singleton
      cell, then returns the set {(i, j)};
    - otherwise, returns the empty set."""
    c = self.m[i][j]
    n = len(c)
    c.discard(x)
    self.m[i][j] = c
    if len(c) == 0:
        raise Unsolvable()
    return {(i, j)} if 1 == len(c) < n else set()

Sudoku.ruleout = sudoku_ruleout


In [ ]:
def sudoku_propagate_cell(self, ij):
    """Propagates the singleton value at cell (i, j), returning the list
    of newly-singleton cells."""
    i, j = ij
    if len(self.m[i][j]) > 1:
        # Nothing to propagate from cell (i,j).
        return {}
    # We keep track of the newly-singleton cells.
    newly_singleton = set()
    x = getel(self.m[i][j]) # Value at (i, j).
    # Same row.
    for jj in range(9):
        if jj != j: # Do not propagate to the element itself.
            newly_singleton.update(self.ruleout(i, jj, x))
    # TODO: Do the same for the same row, column
    return newly_singleton

Sudoku.propagate_cell = sudoku_propagate_cell


In [ ]:
### Exercise: define cell propagation

def sudoku_propagate_cell(self, ij):
    """Propagates the singleton value at cell (i, j), returning the list
    of newly-singleton cells."""
    i, j = ij
    if len(self.m[i][j]) > 1:
        # Nothing to propagate from cell (i,j).
        return set()
    # We keep track of the newly-singleton cells.
    newly_singleton = set()
    x = getel(self.m[i][j]) # Value at (i, j).
    # Same row.
    for jj in range(9):
        if jj != j: # Do not propagate to the element itself.
            newly_singleton.update(self.ruleout(i, jj, x))
    # Same column.
    ### BEGIN SOLUTION
    raise NotImplementedError()
    ### END SOLUTION
    # Same block of 3x3 cells.
    ### BEGIN SOLUTION
    raise NotImplementedError()
    ### END SOLUTION
    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.propagate_cell = sudoku_propagate_cell


In [ ]:
### Tests for cell propagation

tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
try:
    tsd.propagate_cell((0, 2))
except Unsolvable:
    print("Good! It was unsolvable.")
else:
    raise Exception("Hey, it was unsolvable")

tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2, 3]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
assert_equal(tsd.propagate_cell((0, 2)), {(0, 8), (2, 0)})



+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ _2_______|_____6___ ______7__ _______8_|________9 12_4_____ _2_______|
|_____6___ ______7__ 12_4__7__|123______ ________9 ____5____|__3______ 12_4_____ _______8_|
|12_______ ________9 _______8_|__3______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|1_____7_9 _____6___ 1__4__7_9|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 _______8_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

### Propagating all cells, once

The simplest thing we can do is propagate each cell, once. 

In [ ]:
def sudoku_propagate_all_cells_once(self):
    """This function propagates the constraints from all singletons."""
    for i in range(9):
        for j in range(9):
            self.propagate_cell((i, j))

Sudoku.propagate_all_cells_once = sudoku_propagate_all_cells_once


In [ ]:
sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.propagate_all_cells_once()
sd.show()
sd.show(details=True)


+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|853|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|..7|284|
|...|419|.35|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|_2___6___ ______7__ _2_4_6_8_|1__4___89 12_4____9 _2_4___8_|
|_____6___ _2_4__7__ _2_4__7__|1________ ________9 ____5____|__34__78_ _234_____ _2_4__78_|
|12_______ ________9 _______8_|_23______ __34_____ _2_4_____|1_345_7__ _____6___ _2_4__7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 12__5____ 12__5___9|____5_7_9 _____6___ 1__4__7__|___45_7_9 _2_45___9 __3______|
|___4_____ _2__5____ _2__56__9|_______8_ ____5____ __3______|____5_7_9 _2__5___9 1__

### Propagating all cells, repeatedly

This is a good beginning, but it's not quite enough. 
As we propagate the constraints, cells that did not use to be singletons may have become singletons.  For eample, in the above example, the center cell has become known to be a 5: we need to make sure that also these singletons are propagated. 

This is why we have written propagate_cell so that it returns the set of newly-singleton cells.  
We need now to write a method full_propagation that at the beginning starts with a set of _to_propagate_ cells (if it is not specified, then we just take it to consist of all singleton cells).  Then, it picks a cell from the to_propagate set, and propagates from it, adding any newly singleton cell to to_propagate.  Once there are no more cells to be propagated, the method returns. 
If this sounds similar to graph reachability, it is ... because it is!  It is once again the algorithmic pattern of keeping a list of work to be done, then iteratively picking an element from the list, doing it, possibly updating the list of work to be done with new work that has to be done as a result of what we just did, and continuing in this fashion until there is nothing left to do. 
We will let you write this function.  The portion you have to write can be done in three lines of code.

In [ ]:
### Exercise: define full propagation

def sudoku_full_propagation(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all
    newly discovered singleton cells, until no more propagation is possible."""
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    # This code is the (A) code; will be referenced later.
    ### BEGIN SOLUTION
    raise NotImplementedError()
    ### END SOLUTION

Sudoku.full_propagation = sudoku_full_propagation


In [ ]:
### Tests for full propagation

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.full_propagation()
sd.show(details=True)
sdd = Sudoku.from_string('[[[5], [3], [4], [6], [7], [8], [9], [1], [2]], [[6], [7], [2], [1], [9], [5], [3], [4], [8]], [[1], [9], [8], [3], [4], [2], [5], [6], [7]], [[8], [5], [9], [7], [6], [1], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[9], [6], [1], [5], [3], [7], [2], [8], [4]], [[2], [8], [7], [4], [1], [9], [6], [3], [5]], [[3], [4], [5], [2], [8], [6], [1], [7], [9]]]')
assert_equal(sd, sdd)



+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ ___4_____|_____6___ ______7__ _______8_|________9 1________ _2_______|
|_____6___ ______7__ _2_______|1________ ________9 ____5____|__3______ ___4_____ _______8_|
|1________ ________9 _______8_|__3______ ___4_____ _2_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|______7__ _____6___ 1________|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|________9 _____6___ 1________|____5____ __3______ ______7__|_2_______ _______8_ ___4_____|
|_2_______ _______8_ ______7__|___4_____ 1________ ________9|_____6___ __3______

We solved our example problem!  Constraint propagation, iterated, led us to the solution!

## Searching for a solution

Many Sudoku problems can be solved entirely by constraint propagation.  
They are designed to be so: they are designed to be relatively easy, so that humans can solve them while on a lounge chair at the beach -- I know this from personal experience! 

But it is by no means necessary that this is true. 
If we create more complex problems, or less determined problems, constraint propagation no longer suffices. 
As a simple example, let's just blank some cells in the previous problem, and run full propagation again:

In [ ]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
sd.show()
sd.full_propagation()
sd.show(details=True)


+---+---+---+
|53.|.7.|...|
|6..|.95|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|41.|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+---------------------

As we see, there are still undetermined values.  We can peek into the detailed state of the solution:


In [ ]:
sd.show(details=True)
# Let's save this Sudoku for later.
sd_partially_solved = Sudoku(sd)


+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 ______78_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

What can we do when constraint propagation fails? 
The only thing we can do is make a guess.  We can take one of the cells whose set contains multiple digits, such as cell (2, 0) (starting counting at 0, as in Python), which contains $\{1, 2\}$, and try one of the values, for instance $1$.  
We can see whether assigning to the cell the singleton set $\{1\}$ leads to the solution. 
If not, we try the value $\{2\}$ instead. 
If the Sudoku problem has a solution, one of these two values must work. 

Classically, this way of searching for a solution has been called search with _backtracking._
The backtracking is because we can choose a value, say $1$, and then do a lot of work, propagating the new constraint, making further guesses, and so on and so forth.  If that does not pan out, we must "backtrack" and return to our guess, choosing (in our example) $2$ instead. 

Let us implement search with backtracking.  What we need to do is something like this: 

search():
1. propagate constraints.
1. if solved, hoorrayy!
1. if impossible, raise Unsolvable()
1. if not fully solved, pick a cell with multiple digits possible, and iteratively:
 * Assign one of the possible values to the cell. 
 * Call search() with that value for the cell.
 * If Unsolvable is raised by the search() call, move on to the next value.
 * If all values returned Unsolvable (if we tried them all), then we raise Unsolvable.

So we see that search() is a recursive function.  
From the pseudo-code above, we see that it might be better to pick a cell with few values possible at step 4 above, so as to make our chances of success as good as possible.  For instance, it is much better to choose a cell with set $\{1, 2\}$ than one with set $\{1, 3, 5, 6, 7, 9\}$, as the probability of success is $1/2$ in the first case and $1/6$ in the second case. 
Of course, it may be possible to come up with much better heuristics to guide our search, but this will have to do so far. 

One fine point with the search above is the following.  So far, an object has a self.m matrix, which contains the status of the Sudoku solution. 
We cannot simply pass self.m recursively to search(), because in the course of the search and constraint propagation, self.m will be modified, and there is no easy way to keep track of these modifications. 
Rather, we will write search() as a method, and when we call it, we will:

* First, create a copy of the current object via the Sudoku constructor, so we have a copy we can modify. 
* Second, we assign one of the values to the cell, as above; 
* Third, we will call the search() method of that object. 

Furthermore, when a solution is found, as in the hoorraay! above, we need to somehow return the solution. 
There are two ways of doing this: via standard returns, or by raising an exception. 


In [ ]:
def sudoku_done(self):
    """Checks whether an instance of Sudoku is solved."""
    for i in range(9):
        for j in range(9):
            if len(self.m[i][j]) > 1:
                return False
    return True

Sudoku.done = sudoku_done


def sudoku_search(self, new_cell=None):
    """Tries to solve a Sudoku instance."""
    to_propagate = None if new_cell is None else {new_cell}
    self.full_propagation(to_propagate=to_propagate)
    if self.done():
        return self # We are a solution
    # We need to search.  Picks a cell with as few candidates as possible.
    candidates = [(len(self.m[i][j]), i, j)
                   for i in range(9) for j in range(9) if len(self.m[i][j]) > 1]
    _, i, j = min(candidates)
    values = self.m[i][j]
    # values contains the list of values we need to try for cell i, j.
    # print("Searching values", values, "for cell", i, j)
    for x in values:
        # print("Trying value", x)
        sd = Sudoku(self)
        sd.m[i][j] = {x}
        try:
            # If we find a solution, we return it.
            return sd.search(new_cell=(i, j))
        except Unsolvable:
            # Go to next value.
            pass
    # All values have been tried, apparently with no success.
    raise Unsolvable()

Sudoku.search = sudoku_search


def sudoku_solve(self, do_print=True):
    """Wrapper function, calls self and shows the solution if any."""
    try:
        r = self.search()
        if do_print:
            print("We found a solution:")
            r.show()
            return r
    except Unsolvable:
        if do_print:
            print("The problem has no solutions")

Sudoku.solve = sudoku_solve


Let us try this on our previous Sudoku problem that was not solvable via constraint propagation alone.

In [ ]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
sd.solve()


We found a solution:
+---+---+---+
|531|678|942|
|674|295|318|
|298|341|567|
+---+---+---+
|859|167|423|
|426|853|791|
|713|924|856|
+---+---+---+
|165|739|284|
|987|412|635|
|342|586|179|
+---+---+---+


It works, search with constraint propagation solved the Sudoku puzzle!

## The choice - constraint propagation - recursion paradigm.

We have learned a general strategy for solving difficult problems.  The strategy can be summarized thus: **choice - constraint propagation - recursion.** 

In the _choice_ step, we make one guess from a set of possible guesses.  If we want our search for a solution to be exhaustive, as in the above Sudoku example, we ensure that we try iteratively all choices from a set of choices chosen so that at least one of them must succeed.  In the above example, we know that at least one of the digit values must be the true one, hence our search is exhaustive.  In other cases, we can trade off exhaustiveness for efficiency, and we may try only a few choices, guided perhaps by an heuristic. 

The _constraint propagation_ step propagates the consequences of the choice to the problem.  Each choice thus gives rise to a new problem, which is a little bit simpler than the original one as some of the possible choices, that is, some of its complexity, has been removed.  In the Sudoku case, the new problem has less indetermination, as at least one more of its cells has a known digit in it. 

The problems resulting from _constraint propagation_, while simpler, may not be solved yet.  Hence, we _recur_, calling the solution procedure on them as well.  As these problems are simpler (they contain fewer choices), eventually the recursion must reach a point where no more choice is possible, and whether constraint propagation should yield a completely defined problem, one of which it is possible to say whether it is solvable or not with a trivial test.  This forms the base case for the recursion. 

This solution strategy applies very generally, to problems well beyond Sudoku.

## Part 2: Digits must go somewhere

If you have played Sudoku before, you might have found the way we solved Sudoku puzzles a bit odd. 
The constraint we encoded is: 

> If a digit appears in a cell, it cannot appear anywhere else on the same row, column, or 3x3 block as the cell. 

This _is_ a rule of Sudoku.  Normally, however, we hear Sudoku described in a different way:

> Every column, row, and 3x3 block should contain all the 1...9 digits exactly once.

There are two questions.  The first is: are the two definitions equivalent? 
Well, no; the first definition does not say what the digits are (e.g., does not rule out 0).  But in our Sudoku representation, we _start_ by saying that every cell can contain only one of 1...9.  If every row (or column, or 3x3 block) cannot contain more than one repetition of each digit, and if there are 9 digits and 9 cells in the row (or column, or block), then clearly every digit must appear exactly once in the row (or column, or block).  So once the set of digits is specified, the two definitions are equivalent. 

The second question is: but still, what happens to the method we usually employ to solve Sudoku? 
I generally don't solve Sudoku puzzles by focusing on one cell at a time, and thinking: is it the case that this call can contain only one digit? 
This is the strategy employed by the solver above.  But it is not the strategy I normally use. 
I generally solve Sudoku puzzles by looking at a block (or row, or column), and thinking: let's consider the digit $k$ ($1 \leq k \leq 9$).  Where can it go in the block?  And if I find that the digit can go in one block cell only, I write it there.  
Does the solver work even without this "where can it go" strategy?  And can we make it follow it? 

The solver works even without the "where can it go" strategy because it exaustively tries all possibilities.  This means the solver works without the strategy; it does not say that the solver works _well_ without the strategy. 

We can certainly implement the _where can it go_ strategy, as part of constraint propagation; it would make our solver more efficient. 


### Adding the where can it go heuristics

There is a subtle point in applying the _where can it go_ heuristics. 

Before, when our only constraint was the uniqueness in each row, column, and block, we needed to propagate only from cells that hold a singleton value. 
If a cell held a non-singleton set of digits, such as $\{2, 5\}$, no values could be ruled out as a consequence of this on the same row, column, or block. 

The _where can it go_ heuristic, instead, benefits from knowing that in a cell, the set of values went for instance from $\{2, 3, 5\}$ to $\{2, 5\}$: by ruling out the possibility of a $3$ in this cell, it may be possibe to deduct that the digit $3$ can appear in only one (other) place in the block, and place it there. 

Thus, we may be tempted to rewrite the code, and include in the _to_propagate_ list of cells all cells whose set of possible values has shrunk. 
This may lead, however, to an inefficient implementation.  When we modify one cell, there are up to 8 + 8 + 4 = 20 other cells whose values might have changed. 
We believe it is more efficient to first do propagation as before, based on singletons, and then apply the _where can it go_ heuristics on the whole Sudoku board. 
The _where can it go_ heuristic will return a (possibly empty) set of cells which have become singletons, and we can then propagate these. 

Thus, we replace the _full_propagation_ method previously defined with this new one, where the (A) block of code is what you previously wrote in _full_propagation_.


In [ ]:
### Exercise: define full propagation with where can it go

def sudoku_full_propagation_with_where_can_it_go(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all
    newly discovered singleton cells, until no more propagation is possible."""
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    while len(to_propagate) > 0:
        # Here is your previous solution code from (A) in full_propagation.
        # Please copy it below.
        ### BEGIN SOLUTION
        raise NotImplementedError()
        ### END SOLUTION

        # Now we check whether there is any other propagation that we can
        # get from the where can it go rule.
        to_propagate = self.where_can_it_go()



To implement the _where_can_it_go_ method, let us write a helper function, or better, let's have you write it.  Given a sequence of sets $S_1, S_2, \ldots, S_n$, we want to obtain the set of elements that appear in _exactly one_ of the sets (that is, they appear in one set, and _only_ in one set).   Mathematically, we can write this as 
$$
(S_1 \setminus (S_2 \cup \cdots \cup S_n)) \cup (S_2 \setminus (S_1 \cup S_3 \cup \cdots \cup S_n)) \cup \cdots \cup
(S_n \setminus (S_1 \cup \cdots \cup S_{n-1}))
$$
even though that's certainly not the easiest way to compute it!
The problem can be solved with the help of [defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict!) to count the occurrences, and is 5 lines long.


In [ ]:
### Exercise: define helper function to check once-only occurrence

from collections import defaultdict

def occurs_once_in_sets(set_sequence):
    """Returns the elements that occur only once in the sequence of sets set_sequence.
    The elements are returned as a set."""
    ### BEGIN SOLUTION
    raise NotImplementedError()
    ### END SOLUTION


Let us test it.

In [ ]:
### Tests for once-only

from nose.tools import assert_equal

assert_equal(occurs_once_in_sets([{1, 2}, {2, 3}]), {1, 3})



We are now ready to write -- or better, to have you write -- the _where_can_it_go_ method.  
The method is global: it examines all rows, all columns, and all blocks.  
If it finds that in a row (or column, or block), a value can fit in only one cell, and that cell is not currently a singleton (for otherwise there is nothing to be done), it sets the value in the cell, and it adds the cell to the newly_singleton set that is returned, just as in propagate_cell. 
The portion of method that you need to write is about two dozen lines of code long.

In [ ]:
# @title Exercise: write where_can_it_go

def sudoku_where_can_it_go(self):
    """Sets some cell values according to the where can it go
    heuristics, by examining all rows, colums, and blocks."""
    newly_singleton = set()

    ### BEGIN SOLUTION
    raise NotImplementedError()
    ### END SOLUTION

    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.where_can_it_go = sudoku_where_can_it_go


Let us test it.  We cannot test this code in one iteration only, since its result may depend on the order in which you apply the method to rows and columns. 
Rather, we apply the method until it can determine no more cell values.

In [ ]:
### Tests for where can it go

sd = Sudoku.from_string('[[[5], [3], [1, 2, 4], [1, 2, 6], [7], [1, 2, 6, 8], [4, 8, 9], [1, 2, 4], [2, 8]], [[6], [1, 4, 7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3, 4, 8], [1, 2, 4], [2, 7, 8]], [[1, 2], [9], [8], [1, 2, 3], [4], [1, 2], [3, 5], [6], [2, 7]], [[8], [1, 5], [1, 5, 9], [1, 7, 9], [6], [1, 4, 7, 9], [4, 5], [2, 4, 5], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1, 5], [1, 3, 5, 9], [1, 9], [2], [1, 4, 9], [4, 5, 8], [4, 5], [6]], [[1, 9], [6], [1, 5, 7, 9], [5, 7, 9], [3], [7, 9], [2], [8], [4]], [[2, 9], [7, 8], [2, 7, 9], [4], [1], [2, 7, 9], [6], [3], [5]], [[2, 3], [4, 5], [2, 3, 4, 5], [2, 5, 6], [8], [2, 6], [1], [7], [9]]]')
print("Original:")
sd.show(details=True)
new_singletons = set()
while True:
    new_s = sd.where_can_it_go()
    if len(new_s) == 0:
        break
    new_singletons |= new_s
assert_equal(new_singletons,
             {(3, 2), (2, 6), (7, 1), (5, 6), (2, 8), (8, 0), (0, 5), (1, 6),
              (2, 3), (3, 7), (0, 3), (5, 1), (0, 8), (8, 5), (5, 3), (5, 5),
              (8, 1), (5, 7), (3, 1), (0, 6), (1, 8), (3, 6), (5, 2), (1, 1)})
print("After where can it go:")
sd.show(details=True)
sdd = Sudoku.from_string('[[[5], [3], [1, 2, 4], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
print("The above should be equal to:")
sdd.show(details=True)
assert_equal(sd, sdd)

sd = Sudoku([
    '___26_7_1',
    '68__7____',
    '1____45__',
    '82_1___4_',
    '__46_2___',
    '_5___3_28',
    '___3___74',
    '_4__5__36',
    '7_3_18___'
])
print("Another Original:")
sd.show(details=True)
print("Propagate once:")
sd.propagate_all_cells_once()
# sd.show(details=True)
new_singletons = set()
while True:
    new_s = sd.where_can_it_go()
    if len(new_s) == 0:
        break
    new_singletons |= new_s
print("After where can it go:")
sd.show(details=True)
sdd = Sudoku.from_string('[[[4], [3], [5], [2], [6], [9], [7], [8], [1]], [[6], [8], [2], [5], [7], [1], [4], [9], [3]], [[1], [9], [7], [8], [3], [4], [5], [6], [2]], [[8], [2], [6], [1], [9], [5], [3], [4], [7]], [[3], [7], [4], [6], [8], [2], [9], [1], [5]], [[9], [5], [1], [7], [4], [3], [6], [2], [8]], [[5], [1], [1, 2, 5, 6, 8, 9], [3], [2], [6], [1, 2, 8, 9], [7], [4]], [[2], [4], [1, 2, 8, 9], [9], [5], [7], [1, 2, 8, 9], [3], [6]], [[7], [6], [3], [4], [1], [8], [2], [5], [9]]]')
print("The above should be equal to:")
sdd.show(details=True)
assert_equal(sd, sdd)



Original:
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 ______78_ _2____7_9|___4_____ 1________ _2____7_9|_____6___

Let us try it now on a real probem. Note from before that this Sudoku instance could not be solved via propagate_cells alone:

In [ ]:
sd = Sudoku(sd_partially_solved)
newly_singleton = sd.where_can_it_go()
print("Newly singleton:", newly_singleton)
print("Resulting Sudoku:")
sd.show(details=True)


Newly singleton: {(3, 2), (2, 6), (7, 1), (5, 6), (2, 8), (8, 0), (5, 7), (0, 6), (0, 5), (1, 6), (3, 6), (3, 7), (0, 3), (5, 2)}
Resulting Sudoku:
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|_____6___ ______7__ _______8_|________9 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__3______ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ ________9|1_____7_9 _____6___ 1__4__7_9|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ __3______|1_______9 _2_______ 1__4____9|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1__

As we can see, the heuristics led to substantial progress.   Let us incorporate it in the Sudoku solver. 

In [ ]:
Sudoku.full_propagation = sudoku_full_propagation_with_where_can_it_go


Let us try again to solve a Sudoku example which, as we saw before, could not be solved by constrain propagation only (without using the _where can it go_ heuristics).  Can we solve it now via constraint propagation?

In [ ]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
print("Initial:")
sd.show()
sd.full_propagation()
print("After full propagation with where can it go:")
sd.show()


Initial:
+---+---+---+
|53.|.7.|...|
|6..|.95|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|41.|..5|
|...|.8.|.79|
+---+---+---+
After full propagation with where can it go:
+---+---+---+
|53.|678|9.2|
|67.|.95|3.8|
|.98|34.|567|
+---+---+---+
|859|.6.|423|
|426|853|791|
|713|924|856|
+---+---+---+
|.6.|.3.|284|
|.8.|41.|635|
|34.|.86|179|
+---+---+---+


No!  We still cannot! But if we compare the above with the previous attempt, we see that the heuristic led to much more progress; very few positions still remain to be determined via search. 

## Solving some problems from example sites

Let us see how long it takes us to solve examples found around the Web. 
We consider a few from [this site](https://dingo.sbs.arizona.edu/~sandiway/sudoku/examples.html).

In [ ]:
import time


### Daily Telegraph January 19th "Diabolical"



In [ ]:
sd = Sudoku([
    '_2_6_8___',
    '58___97__',
    '____4____',
    '37____5__',
    '6_______4',
    '__8____13',
    '____2____',
    '__98___36',
    '___3_6_9_'
])
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|123|678|945|
|584|239|761|
|967|145|328|
+---+---+---+
|372|461|589|
|691|583|274|
|458|792|613|
+---+---+---+
|836|924|157|
|219|857|436|
|745|316|892|
+---+---+---+
Solved in 0.005837202072143555 seconds


### Vegard Hanssen puzzle 2155141

In [ ]:
sd = Sudoku([
    '___6__4__',
    '7____36__',
    '____91_8_',
    '_________',
    '_5_18___3',
    '___3_6_45',
    '_4_2___6_',
    '9_3______',
    '_2____1__'
])
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|581|672|439|
|792|843|651|
|364|591|782|
+---+---+---+
|438|957|216|
|256|184|973|
|179|326|845|
+---+---+---+
|845|219|367|
|913|768|524|
|627|435|198|
+---+---+---+
Solved in 0.08706307411193848 seconds


### Challenge 1 from Sudoku Solver by Logic

In [ ]:
sd = Sudoku([
    '_2_______',
    '___6____3',
    '_74_8____',
    '_____3__2',
    '_8__4__1_',
    '6__5_____',
    '____1_78_',
    '5____9___',
    '_______4_'
])
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|126|437|958|
|895|621|473|
|374|985|126|
+---+---+---+
|457|193|862|
|983|246|517|
|612|578|394|
+---+---+---+
|269|314|785|
|548|769|231|
|731|852|649|
+---+---+---+
Solved in 0.03747749328613281 seconds


### A very difficult one from 7Sudoku.com

From [here](http://www.7sudoku.com/view-puzzle?date=20190616).

In [ ]:
sd = Sudoku([
    '__58_12__',
    '____4___1',
    '_7_____45',
    '______38_',
    '58_____72',
    '_92______',
    '26_____9_',
    '3___1____',
    '__83_96__'
])
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|435|871|269|
|826|945|731|
|971|236|845|
+---+---+---+
|714|592|386|
|583|164|972|
|692|783|514|
+---+---+---+
|267|458|193|
|359|617|428|
|148|329|657|
+---+---+---+
Solved in 0.007710933685302734 seconds


### The World's Hardest Sudoku

Or so claims Arto Inkala [source](https://www.telegraph.co.uk/news/science/science-news/9359579/Worlds-hardest-sudoku-can-you-crack-it.html).  See [also this](http://www.sudokuwiki.org/Arto_Inkala_Sudoku).  I am not quite sure why they claim that there are unsolvable puzzles; as you see, our solver does pretty well on all of them.

In [ ]:
sd = Sudoku([
    '8________',
    '__36_____',
    '_7__9_2__',
    '_5___7___',
    '____457__',
    '___1___3_',
    '__1____68',
    '__85___1_',
    '_9____4__'
])
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|812|753|649|
|943|682|175|
|675|491|283|
+---+---+---+
|154|237|896|
|369|845|721|
|287|169|534|
+---+---+---+
|521|974|368|
|438|526|917|
|796|318|452|
+---+---+---+
Solved in 0.13454437255859375 seconds


This _was_ hard; it took us almost four times longer than any of the other examples.

### A supposedly even harder one

[source](http://www.sudokuwiki.org/Weekly_Sudoku.asp?puz=28)

In [ ]:
sd = Sudoku([
    '6____894_',
    '9____61__',
    '_7__4____',
    '2__61____',
    '______2__',
    '_89__2___',
    '____6___5',
    '_______3_',
    '8____16__'
])
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|625|178|943|
|948|326|157|
|371|945|862|
+---+---+---+
|257|619|384|
|463|587|291|
|189|432|576|
+---+---+---+
|792|863|415|
|516|294|738|
|834|751|629|
+---+---+---+
Solved in 0.34685206413269043 seconds


Even harder!  But we still made it within one second.

The point is that these puzzles are designed so that they are solvable in an interesting way using a few heuristics.  They are no match for our brute force approach.

## Trying puzzles in bulk

Let us try the puzzles found at [https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt](https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt); apparently lines 517 and 6361 are very hard). 

In [ ]:
import requests

r = requests.get("https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt")
puzzles = r.text.split()


In [ ]:
puzzles[517]


'000000061350000000400050000020000800000601000000700000000080200600400000007000010'

Let us convert these puzzles to our format.

In [ ]:
def convert_to_our_format(s):
    t = s.replace('0', '_')
    r = []
    for i in range(9):
        r.append(t[i * 9: (i + 1) * 9])
    return r


Let us solve those two apparently hard puzzles.

In [ ]:
sd = Sudoku(convert_to_our_format(puzzles[517]))
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|289|374|561|
|351|869|724|
|476|152|983|
+---+---+---+
|124|593|876|
|738|621|495|
|965|748|132|
+---+---+---+
|513|986|247|
|692|417|358|
|847|235|619|
+---+---+---+
Solved in 0.00682520866394043 seconds


In [ ]:
sd = Sudoku(convert_to_our_format(puzzles[6361]))
t = time.time()
sd.solve()
print("Solved in", time.time() - t, "seconds")


We found a solution:
+---+---+---+
|795|681|234|
|416|352|978|
|832|794|156|
+---+---+---+
|964|823|517|
|128|576|493|
|357|419|862|
+---+---+---+
|571|238|649|
|249|167|385|
|683|945|721|
+---+---+---+
Solved in 0.01313018798828125 seconds


In [ ]:
# Let's find the hardest puzzle.
t = 0
max_d = 0.
max_i = None
for i, s in enumerate(puzzles):
    p = convert_to_our_format(puzzles[i])
    sd = Sudoku(p)
    t = time.time()
    sd.solve(do_print=False)
    d = time.time() - t
    if d > max_d:
        max_d = d
        max_i = i
        print("Hardest:", i, "d:", d)
print("The hardest problem is:", max_i)
print("Time:", max_d)



Hardest: 0 d: 0.0030603408813476562
Hardest: 2 d: 0.004228830337524414
Hardest: 4 d: 0.007424354553222656
Hardest: 7 d: 0.016331195831298828
Hardest: 27 d: 0.02524542808532715
Hardest: 46 d: 0.025329113006591797
Hardest: 51 d: 0.025599241256713867
Hardest: 143 d: 0.04078960418701172
Hardest: 195 d: 0.06332707405090332
Hardest: 196 d: 0.1467299461364746
Hardest: 271 d: 0.26159000396728516
Hardest: 1258 d: 0.7234883308410645
Hardest: 5302 d: 1.112680435180664
Hardest: 10980 d: 1.6263811588287354
Hardest: 25632 d: 1.7625515460968018
Hardest: 48287 d: 2.2216758728027344
The hardest problem is: 48287
Time: 2.2216758728027344


In [ ]:
puzzles[48287]


'000000209300000010400000000000301000000600400000400500025080000070050000000000040'